In [ ]:
region = 'us-east-2'
endpoint='ENDPOINT'
threshold = 0.5
bucket = 'BUCKET'
prefix = 'kubeflow-batch/in'

In [ ]:
import boto3
client = boto3.client('sagemaker-runtime', region_name=region)

In [ ]:
import numpy as np
test_data = np.load('test_data.npy')
test_labels = np.load('test_labels.npy')

We need a JSON serializer, so let's reuse one from the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/predictor.py).  I've copied relevant bits of the code below, but we could install the SageMaker Python module and import it instead.

In [ ]:
import json
import six

def _ndarray_to_list(data):
    """
    Args:
        data:
    """
    return data.tolist() if isinstance(data, np.ndarray) else data

def _json_serialize_from_buffer(buff):
    """
    Args:
        buff:
    """
    return buff.read()

def json_serial(data):

    if isinstance(data, dict):
        # convert each value in dict from a numpy array to a list if necessary, so they can be
        # json serialized
        return json.dumps({k: _ndarray_to_list(v) for k, v in six.iteritems(data)})

    # files and buffers
    if hasattr(data, "read"):
        return _json_serialize_from_buffer(data)

    return json.dumps(_ndarray_to_list(data))

In [ ]:
def predict(data):
    
    response = client.invoke_endpoint(
            EndpointName = endpoint,
            Body=json_serial(data),
            ContentType='application/json'
        )
    body = response['Body'].read().decode()
    j = json.loads(body)
    return j['predictions'][0][0]

In [ ]:
preds = []
for i in range(0, len(test_labels)):
    pred = predict(test_data[i])
    label = test_labels[i]
    if pred > threshold:
        pred = 1
    else:
        pred = 0
    preds.append(pred)

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix,classification_report
confusion_matrix(preds, test_labels)

In [ ]:
print(classification_report(preds, test_labels))

## Transform to JSON

In preparation for the next section, we're going to save our data in JSON format.

In [ ]:
json_records = []
for i in range(0, len(test_labels)):
    j = json_serial(test_data[i])
    json_records.append(j)

In [ ]:
with open("batch_records.json", "w") as json_file:
    for j in json_records:
        json_file.write(j)
        json_file.write("\n")

In [ ]:
import os

s3 = boto3.client('s3')
s3.upload_file('batch_records.json', bucket, os.path.join(prefix, 'batch_records.json'))